# Notes

## Python/GraphQL libs

[Ariadne](https://pypi.org/project/ariadne/)

- Pronounced "R-E-ad-knee", from greek mythology 🤷
- 1.7k stars, 45 contributors on [GitHub](https://github.com/mirumee/ariadne), 

[Graphene](https://pypi.org/project/graphene/)

- Easy to pronounce, named after super rad game-changing material
- 7.1k stars, 162 contributors on [GitHub](https://github.com/graphql-python/graphene)
- Let's go with this one...

## Python

3.10.2, and I'll try to use a walrus operator in the name of Guido :=

## ORM

SQLAlchemy

- Industry standard from my experience
- Peewee sounds cool, but I haven't used it. Maybe next time!


## Database setup

SQLite

- though this should work just about the same with any database